In [1]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import deque
import random
import os

In [2]:
# Hyperparameters
GAMMA = 0.99
TAU = 0.005
LR_ACTOR = 0.0001
LR_CRITIC = 0.001
BUFFER_SIZE = 1000000
BATCH_SIZE = 64
NOISE_DECAY = 0.99
MAX_EPISODES = 1000
MAX_TIMESTEPS = 2000
SAVE_PATH = "./ddpg_bipedalwalker"


In [3]:
# Set the random seed for reproducibility
SEED = 32
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
env = gym.make('BipedalWalker-v3')
env.seed(SEED)

/home/janak/Documents/Pytorch_CPU/venv/lib/python3.10/site-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/janak/Documents/Pytorch_CPU/venv/lib/python3.10/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/janak/Documents/Pytorch_CPU/venv/lib/python3.10/site-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


[32]

In [4]:
# Actor Network
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, max_action):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(state_dim, 400)
        self.fc2 = nn.Linear(400, 300)
        self.fc3 = nn.Linear(300, action_dim)
        self.max_action = max_action

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        return x * self.max_action

# Critic Network
class Critic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(state_dim + action_dim, 400)
        self.fc2 = nn.Linear(400, 300)
        self.fc3 = nn.Linear(300, 1)

    def forward(self, state, action):
        x = torch.cat([state, action], 1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Replay Buffer
class ReplayBuffer:
    def __init__(self, max_size):
        self.buffer = deque(maxlen=max_size)

    def add(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones)

    def size(self):
        return len(self.buffer)

# DDPG Agent
class DDPGAgent:
    def __init__(self, state_dim, action_dim, max_action):
        self.actor = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=LR_ACTOR)

        self.critic = Critic(state_dim, action_dim).to(device)
        self.critic_target = Critic(state_dim, action_dim).to(device)
        self.critic_target.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=LR_CRITIC)

        self.replay_buffer = ReplayBuffer(BUFFER_SIZE)
        self.max_action = max_action
        self.noise = 0.1

    def select_action(self, state):
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        return self.actor(state).cpu().data.numpy().flatten()

    def train(self):
        if self.replay_buffer.size() < BATCH_SIZE:
            return

        states, actions, rewards, next_states, dones = self.replay_buffer.sample(BATCH_SIZE)

        states = torch.FloatTensor(states).to(device)
        actions = torch.FloatTensor(actions).to(device)
        rewards = torch.FloatTensor(rewards).unsqueeze(1).to(device)
        next_states = torch.FloatTensor(next_states).to(device)
        dones = torch.FloatTensor(dones).unsqueeze(1).to(device)

        # Critic loss
        target_actions = self.actor_target(next_states)
        target_q = self.critic_target(next_states, target_actions)
        target_q = rewards + ((1 - dones) * GAMMA * target_q).detach()

        current_q = self.critic(states, actions)
        critic_loss = nn.MSELoss()(current_q, target_q)

        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Actor loss
        actor_loss = -self.critic(states, self.actor(states)).mean()

        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # Update target networks
        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
            target_param.data.copy_(TAU * param.data + (1 - TAU) * target_param.data)

        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
            target_param.data.copy_(TAU * param.data + (1 - TAU) * target_param.data)

    def add_to_replay_buffer(self, state, action, reward, next_state, done):
        self.replay_buffer.add(state, action, reward, next_state, done)

    def save(self, filename):
        torch.save(self.actor.state_dict(), filename + "_actor.pth")
        torch.save(self.critic.state_dict(), filename + "_critic.pth")

    def load(self, filename):
        self.actor.load_state_dict(torch.load(filename + "_actor.pth"))
        self.critic.load_state_dict(torch.load(filename + "_critic.pth"))



In [5]:
# Initialize environment and agent
# env = gym.make('BipedalWalker-v3')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

agent = DDPGAgent(state_dim, action_dim, max_action)

In [6]:
episode_rewards = []

In [7]:


# Training loop
for episode in range(MAX_EPISODES):
    state = env.reset()
    episode_reward = 0
    agent.noise *= NOISE_DECAY

    for t in range(MAX_TIMESTEPS):
        action = agent.select_action(state) + np.random.normal(0, agent.noise, size=action_dim)
        action = action.clip(env.action_space.low, env.action_space.high)
        
        next_state, reward, done, _ = env.step(action)
        agent.add_to_replay_buffer(state, action, reward, next_state, done)

        state = next_state
        episode_reward += reward

        agent.train()

        if done:
            break

    print(f"Episode: {episode}, Reward: {episode_reward}")
    episode_rewards.append(episode_reward)

    # Save the model every 10 episodes
    if (episode + 1) % 10 == 0:
        agent.save(SAVE_PATH)

env.close()


/home/janak/Documents/Pytorch_CPU/venv/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode: 0, Reward: -104.74681636101627
Episode: 1, Reward: -112.61592918835186
Episode: 2, Reward: -122.21226414155713
Episode: 3, Reward: -132.23586421377738
Episode: 4, Reward: -125.58899066855001
Episode: 5, Reward: -102.04380804349775
Episode: 6, Reward: -129.0697555354034
Episode: 7, Reward: -108.3844748090671
Episode: 8, Reward: -124.2452241349934
Episode: 9, Reward: -114.6601941691709
Episode: 10, Reward: -109.14364149126148
Episode: 11, Reward: -107.44776705151115
Episode: 12, Reward: -124.66014167003361
Episode: 13, Reward: -140.50777176418677
Episode: 14, Reward: -129.91921054979665
Episode: 15, Reward: -82.39351351050772
Episode: 16, Reward: -124.75079445758698
Episode: 17, Reward: -113.88315727439512
Episode: 18, Reward: -125.98248427167685
Episode: 19, Reward: -117.57510091269506
Episode: 20, Reward: -112.38861185132747
Episode: 21, Reward: -128.14594840927776
Episode: 22, Reward: -123.30861826861235
Episode: 23, Reward: -116.56068815341568
Episode: 24, Reward: -128.55255